In [ ]:
import json
import random
import numpy as np

def stable_softmax(x):
    z = x - max(x)
    numerator = np.exp(z)
    denominator = np.sum(numerator)
    softmax = numerator/denominator

    return softmax

In [ ]:
# for the form exp(x[0]*i)*x[1]
# fit to FBGM team rating coeffcients 
# tweaks 0.25*QB1 for Backup QB, LB smoothed out by hand, backup K and P set to min weight of set
pos_V = {'QB': [-1.396, 0.275],
 'RB': [-1.49, 0.019],
 'TE': [-1.527, 0.031],
 'WR': [-0.524, 0.061],
 'OL': [-0.213, 0.181],
 'CB': [-0.568, 0.136],
 'S': [-0.508, 0.101],
 'LB': [-0.389, 0.072],
 'DL': [-0.286, 0.234],
 'K': [-4.386, 0.08],
 'P': [-3.936, 0.051]}

In [ ]:
with open('FBGM_League_37_2020_re_sign_players.json','r',encoding='utf-8-sig') as fp:
    txt = fp.read()

In [ ]:
data = json.loads(txt)

In [ ]:
data.keys()

In [ ]:
for p in data['players']:
    p['freeAgentMood']= 30*[0]
for t in data['gameAttributes']:
    if t['key'] == 'salaryCap':
        cap = t['value']
    if t['key'] == 'minContract':
        minS = t['value']
    if t['key'] == 'maxContract':
        maxS = t['value']
    if t['key'] == 'season':
        season = t['value']
numTeams = len(data['teams'])

In [ ]:
RAND_NUM = 256 # power of 2 plz
RAND_NOISE = 0.5
RN = RAND_NUM - 1 # mod 2 into hash
rvec = np.random.randn(RAND_NUM)*RAND_NOISE

TEMP = 0.9
LEARNING_RATE = .5
ROUNDS = 60
SCALE_CAP = 1.0 # scaling
ADD_CAP = maxS*0.15 # exception salary
USE_OWN_INIT = True
USE_EXPIRINGS = False
EXP_INIT = .4 # try to get init function to work for both
MIN_BIDS = 1
IS_FOOTBALL = True

In [ ]:
def bbgm_value(team):
    teamvt = [v*np.exp(-0.1733*i)*0.336273 for i,v in enumerate(sorted(team,reverse=True))]
    return sum(teamvt)
bbgm_value([72,55,55,52,56,53,62,78])

In [ ]:
def fbgm_value(team,weights):
    teamvt = [v*np.exp(weights[0]*i)*weights[1] for i,v in enumerate(sorted(team,reverse=True))]
    return sum(teamvt)


In [ ]:
players = {}
for p in data['players']:
    if USE_EXPIRINGS and p['tid'] >= 0:
        if p['contract']['exp'] > season+1:
            continue
    elif p['tid'] != -1:
        continue
    apy = p['contract']['amount']


    val = p['value']
    # could consider cross-pos play by team but won't
    pos = sorted([(v,p) for p,v in p['ratings'][-1]['ovrs'].items() if p not in ['KR','PR']])[-1][1] if IS_FOOTBALL else ''
    if pos == 'QB':
        val /= 1.25
    elif pos in ['P','K']:
        val /= 0.25
    players[p['pid']] = (val,apy,pos)

if True and not IS_FOOTBALL:
    p_keys = list(players.keys())
    p_vals = [players[k][1] for k in p_keys]
    # 0 is highest, etc. 
    order = len(p_vals)-1 - np.argsort(p_vals)

    # set order by ourselves simply based on order and number of teams!
    # i know it's not actually the right intercepts but whatever, it's close. 
    for k,v,o in zip(p_keys,p_vals,order):
        nv = round(maxS*np.exp(-(1+EXP_INIT*USE_EXPIRINGS)*o/numTeams)+minS)
        players[k] = (players[k][0],nv,players[k][2])

In [ ]:
from collections import defaultdict
teams = defaultdict(list)
team_talent = defaultdict(list) if not IS_FOOTBALL else defaultdict(lambda: defaultdict(list))
for p in data['players']:
    if p['tid'] >= 0:
        teams[p['tid']].append(p['contract']['amount'])
        if IS_FOOTBALL:
            pos = sorted([(v,p) for p,v in p['ratings'][-1]['ovrs'].items() if p not in ['KR','PR']])[-1][1]
            team_talent[p['tid']][pos].append(p['value'])
        else:
            team_talent[p['tid']].append(p['value'])

    if p['pid'] in [163,138]:
        print(p['contract']['amount'])


In [ ]:
if IS_FOOTBALL:
    team_talent_total = defaultdict(lambda: defaultdict(list))
    for t in team_talent:
        for p in pos_V:
            team_talent_total[t][p] = fbgm_value(team_talent.get(t,{}).get(p,[]),pos_V[p])

In [ ]:
team_totals = {k:sum(v) for k,v in teams.items()}
for t in data['teams']:
    team_totals[t['tid']] = team_totals.get(t['tid'],0)

In [ ]:
#team_totals = {k:0 for k,v in team_totals.items()}
team_totals

In [ ]:
for i in range(ROUNDS):
    # teams bid on players
    OFFSET = LEARNING_RATE*(1/(1+i/ROUNDS)**3)
    SCALE_UP = 1.0 + OFFSET
    SCALE_DOWN = 1.0 - OFFSET
    bids = defaultdict(int)
    team_data = list(team_totals.items())
    random.shuffle(team_data)
    for t,c in team_data: # randomize team order
        if not IS_FOOTBALL:
            val_o = bbgm_value(team_talent[t])

        budget = SCALE_CAP*max(0,cap-c)+ADD_CAP
        selected = set([pid for pid,p in players.items() if bids[pid] > 1]) # skip players who we know get signed at their current price
        #selected = set()
        while budget > 0:
            # get valid players
            if not IS_FOOTBALL:
                valid_p = [(bbgm_value(team_talent[t] +[p[0]])-val_o,pid) for pid,p in players.items() if p[1]<=budget and pid not in selected]
            else:
                valid_p = []
                for pid,p in players.items():
                    if p[1]> budget or pid in selected:
                        continue
                    val_o = team_talent_total[t].get(p[2],0)
                    tal = team_talent.get(t,{}).get(p[2],[]) 
                    val_n = fbgm_value(tal+[p[0]],pos_V[p[2]])
                    valid_p.append((val_n-val_o,pid))
            if len(valid_p) > 0:
                # sample one
                vpa = np.array(valid_p)
                probs = stable_softmax(vpa[:,0]*TEMP)
                pid = int(np.random.choice(vpa[:,1],1,p=probs))

                # act accordingly
                selected.add(pid)
                bids[pid] = 1 + bids[pid]
                budget -= players[pid][1]
            else:
                break
    print(i,OFFSET,len([_ for _ in players if bids[_] > 1]))

    # players adjust expectations
    for pid,v in players.items():
        if pid in bids and bids[pid] > MIN_BIDS:
            players[pid]  = (v[0],min(maxS,v[1]*SCALE_UP),v[2])
        elif bids[pid] == 0:
            players[pid] = (v[0],max(minS,v[1]*SCALE_DOWN),v[2])
            #print(players[pid],pid)

In [ ]:
res = []
for p in data['players']:
    if p['pid'] not in players:
        continue
    #if p['tid'] != -1:
    #    continue
    res.append((p['contract']['amount'],players[p['pid']][1]))

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(1,2,1)
plt.hist(vpa[:,0])
plt.subplot(1,2,2)
plt.hist(probs)

In [ ]:
plt.scatter(np.array(res)[:,0],np.array(res)[:,1])
plt.xlabel('old contract')
plt.ylabel('new contract')
plt.title('changes')
plt.tight_layout()

In [ ]:
res2 = res

In [ ]:
vs = np.array(list(players.values()))[:,0]
vss = stable_softmax(vs*2.5)
vss *= numTeams*cap*0.75
vss[np.where(vss > maxS)] = maxS
#_ = plt.hist(vss,80)
plt.plot(np.arange(len(vss)),sorted(vss,reverse=True))
plt.xlim(0,100)

In [ ]:
# test the init thing
plt.plot(sorted(np.array(list(players.values()))[:,1],reverse=True))
xn = np.linspace(0,1000,1000)
plt.plot(xn,maxS*np.exp(-(1+EXP_INIT*USE_EXPIRINGS)*xn/(numTeams))+minS)
#plt.xlim(0,100)
plt.ylim(0,maxS*1.2)

In [ ]:
plt.style.use('fivethirtyeight')
plt.scatter(np.array(res)[:,0],np.array(res)[:,1])
plt.xlabel('old contract')
plt.ylabel('new contract')
plt.title('INV_TEMP = {}'.format(TEMP))

In [ ]:
for i,e in enumerate(data['players']):
    if e['pid'] in players:
        if e['pid'] in [163,138]:
            print(int(round(players[e['pid']][1])))

In [ ]:
for i,e in enumerate(data['players']):
    if e['pid'] in players and e['tid'] == -1:
        data['players'][i]['contract']['amount'] = int(round(players[e['pid']][1]))

In [ ]:
with open('fa_fixed.json','w',encoding='utf-8-sig') as fp:
    json.dump(data,fp)

In [ ]:
data['players'][0]